<a href="https://colab.research.google.com/github/aditi25mip10089/Thyroid-Prediction-App/blob/main/Thyroid_Prediction_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

class ThyroidApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Thyroid Recurrence Prediction System")
        self.root.geometry("900x700")
        self.root.configure(bg="#f0f0f0")
        self.root.resizable(False, False)

        # Variables to store encoders and the model
        self.encoders = {}
        self.model = None
        self.feature_columns = [
            'Age', 'Gender', 'Smoking', 'Thyroid Function',
            'Physical Examination', 'Adenopathy', 'Pathology',
            'Focality', 'Risk', 'Stage'
        ]
        # Target variable
        self.target_column = 'Recurred'

        # UI Storage
        self.entries = {}

        # 1. Load and Train Model
        self.load_and_train()

        # 2. Build GUI
        self.create_header()
        self.create_form()
        self.create_buttons()

    def load_and_train(self):
        try:
            # Load Dataset
            df = pd.read_csv('Thyroid_Diff.csv')

            # Preprocessing
            # We need to encode categorical text columns into numbers
            for col in self.feature_columns:
                if col != 'Age': # Age is already numeric
                    le = LabelEncoder()
                    df[col] = le.fit_transform(df[col].astype(str))
                    self.encoders[col] = le # Save encoder to inverse transform later if needed

            # Encode Target
            self.target_encoder = LabelEncoder()
            df[self.target_column] = self.target_encoder.fit_transform(df[self.target_column])

            # Split Data
            X = df[self.feature_columns]
            y = df[self.target_column]

            # Train Model
            self.model = RandomForestClassifier(n_estimators=100, random_state=42)
            self.model.fit(X, y)

            print("Model trained successfully.")

        except FileNotFoundError:
            messagebox.showerror("Error", "Thyroid_Diff.csv not found!\nPlease place the dataset in the same folder.")
            self.root.destroy()
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred during training:\n{e}")
            self.root.destroy()

    def create_header(self):
        header_frame = tk.Frame(self.root, bg="#4a90e2", height=80)
        header_frame.pack(fill=tk.X)

        title_label = tk.Label(
            header_frame,
            text="Thyroid Disease Recurrence Prediction",
            font=("Arial", 24, "bold"),
            bg="#4a90e2",
            fg="white"
        )
        title_label.pack(pady=20)

    def create_form(self):
        form_frame = tk.Frame(self.root, bg="#f0f0f0")
        form_frame.pack(pady=20)

        # Load original data for dropdown options
        try:
            raw_df = pd.read_csv('Thyroid_Diff.csv')
        except:
            raw_df = pd.DataFrame() # Fallback

        row = 0
        col = 0

        for feature in self.feature_columns:
            # Label
            lbl = tk.Label(form_frame, text=feature + ":", font=("Arial", 12, "bold"), bg="#f0f0f0")
            lbl.grid(row=row, column=col, padx=20, pady=10, sticky="w")

            # Input Field
            if feature == 'Age':
                # Entry Box for Age
                ent = tk.Entry(form_frame, font=("Arial", 12), width=22)
                ent.grid(row=row, column=col+1, padx=20, pady=10)
                self.entries[feature] = ent
            else:
                # Combobox for Categorical data
                values = sorted(raw_df[feature].unique().astype(str).tolist()) if not raw_df.empty else []
                combo = ttk.Combobox(form_frame, values=values, font=("Arial", 12), width=20, state="readonly")
                combo.grid(row=row, column=col+1, padx=20, pady=10)
                self.entries[feature] = combo

            # Grid Logic (2 columns layout)
            col += 2
            if col > 2:
                col = 0
                row += 1

    def create_buttons(self):
        btn_frame = tk.Frame(self.root, bg="#f0f0f0")
        btn_frame.pack(pady=20)

        predict_btn = tk.Button(
            btn_frame,
            text="PREDICT RECURRENCE",
            font=("Arial", 14, "bold"),
            bg="#28a745",
            fg="white",
            width=20,
            command=self.predict
        )
        predict_btn.grid(row=0, column=0, padx=20)

        reset_btn = tk.Button(
            btn_frame,
            text="RESET",
            font=("Arial", 14, "bold"),
            bg="#dc3545",
            fg="white",
            width=15,
            command=self.reset
        )
        reset_btn.grid(row=0, column=1, padx=20)

        # Result Label
        self.result_var = tk.StringVar()
        result_display = tk.Label(
            self.root,
            textvariable=self.result_var,
            font=("Arial", 18, "bold"),
            bg="#f0f0f0",
            fg="#333"
        )
        result_display.pack(pady=20)

    def predict(self):
        input_data = []

        try:
            for feature in self.feature_columns:
                val = self.entries[feature].get()

                if not val:
                    messagebox.showwarning("Missing Input", f"Please fill in {feature}")
                    return

                if feature == 'Age':
                    input_data.append(int(val))
                else:
                    # We must encode the input string to the number the model expects
                    # using the encoders we saved during training
                    encoder = self.encoders[feature]
                    encoded_val = encoder.transform([val])[0]
                    input_data.append(encoded_val)

            # Make Prediction
            prediction_idx = self.model.predict([input_data])[0]
            prediction_text = self.target_encoder.inverse_transform([prediction_idx])[0]

            if prediction_text.lower() == 'yes':
                self.result_var.set(f"Prediction: High Risk of Recurrence ({prediction_text})")
                messagebox.showwarning("Result", "Prediction: Disease Recurrence Likely (Yes)")
            else:
                self.result_var.set(f"Prediction: Low Risk of Recurrence ({prediction_text})")
                messagebox.showinfo("Result", "Prediction: No Recurrence Likely (No)")

        except ValueError:
            messagebox.showerror("Input Error", "Age must be a valid number.")
        except Exception as e:
            messagebox.showerror("Error", f"Prediction failed: {e}")

    def reset(self):
        for feature, widget in self.entries.items():
            if isinstance(widget, tk.Entry):
                widget.delete(0, tk.END)
            else:
                widget.set('')
        self.result_var.set("")

if __name__ == "__main__":
    root = tk.Tk()
    app = ThyroidApp(root)
    root.mainloop()